In [1]:
# includes
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import pickle
import os

In [2]:
# params
yr1 = 2013
yr2 = 2022

required = ['Country Code', 'Counterpart Country Code']
years = [str(y) for y in np.arange(yr1, yr2+1)]

In [3]:
# Separate DOT by indicator
df = pd.read_csv('./resouces/downloads/DOT_1960_2022.csv')
df = df[df['Attribute'] == 'Value']
df.drop(['Attribute', 'Indicator Name'], axis=1, inplace=True)

# print(len(df['Country Name'].unique()))
# print(len(df['Counterpart Country Name'].unique()))

In [4]:
correction_dict = {
    'China, P.R.: Mainland'          : 'China',
    'China, P.R.: Hong Kong'         : 'Hong Kong',
    'China, P.R.: Macao'             : 'Macao',

    'Korea, Rep. of'                : 'South Korea',
    'Korea, Dem. People\'s Rep. of' : 'North Korea',

    'Congo, Dem. Rep. of the'       : 'Congo (Kinshasa)',
    'Congo, Rep. of'                : 'Congo (Brazzaville)',

    'Yemen, Rep. of'                : 'Yemen',
    'Yemen, People\'s Dem. Rep. of' : 'South Yemen',
    'Yemen Arab Rep.'               : 'North Yemen',

    'Türkiye, Rep of'               : 'Turkey',
    'Central African Rep.'          : 'Central African Republic',
    'Brunei Darussalam'             : 'Brunei',
    'Côte d\'Ivoire'                : 'Cote d\'Ivoire',
    'Lao People\'s Dem. Rep.'       : 'Laos',
    'Russian Federation'            : 'Russia',
    'Dominican Rep.'                : 'Dominican Republic',
    'Slovak Rep.'                   : 'Slovakia',
    'Gambia, The'                   : 'The Gambia',
    'Syrian Arab Rep.'              : 'Syria',
    'São Tomé and Príncipe, Dem. Rep. of' : 'Sao Tome and Principe',
}

drop_list = {
    'Malaysia, West',
}

for k,v in correction_dict.items():
    df.loc[df['Country Name'] == k, 'Country Name'] = v
    df.loc[df['Counterpart Country Name'] == k, 'Counterpart Country Name'] = v

for c in drop_list:
    df = df[df['Country Name'] != c]
    df = df[df['Counterpart Country Name'] != c]


In [5]:
cname_ccode_map = dict()

if os.path.exists('./resouces/downloads/cname_ccode_map.pkl'):
    print('file already exists')
    with open('./resouces/downloads/cname_ccode_map.pkl', 'rb') as f:
        cname_ccode_map = pickle.load(f)
else:
    print('file does not exists')
    for index, row in df.iterrows():
        name, code = row[0].split(',')[0], row[1]
        if name in cname_ccode_map and cname_ccode_map[name] != code:
            print('1:', name, row[0], code, cname_ccode_map[name], sep='\t')
        cname_ccode_map[name] = code

        name, code = row[3].split(',')[0], row[4]
        if name in cname_ccode_map and cname_ccode_map[name] != code:
            print('2:', name, row[3], code, cname_ccode_map[name], sep='\t\t')
        cname_ccode_map[name] = code

    cname_ccode_map = dict(sorted(cname_ccode_map.items(), key=lambda item: item[1]))

    with open('./resouces/downloads/cname_ccode_map.pkl', 'wb') as f:
        pickle.dump(cname_ccode_map, f)

print('length = ', len(cname_ccode_map))

file does not exists
length =  243


In [8]:
df = df[required+['Indicator Code']+years]
df.head()

,Country Code,Counterpart Country Code,Indicator Code,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,316,186,TXG_FOB_USD,996070,719400,347471,603814,1241903,724256,428837,508305,120484,99749
1,419,746,TMG_CIF_USD,1438390,569667,583545,993794,1781871,1028000,743329,354134,358873,578953
3,419,654,TMG_CIF_USD,NaN,4416,NaN,NaN,NaN,77,NaN,NaN,NaN,NaN
5,913,754,TXG_FOB_USD,267600,94300,56700,2612500,347500,1150400,1058300,31900,2184300,48900
6,913,283,TXG_FOB_USD,771600,645300,2403000,2964300,2226600,6912200,314200,1011300,108500,28100


In [9]:
print('original = ', df.shape)

df.dropna(inplace=True)
print('dropna = ', df.shape)

original =  (97933, 13)
dropna =  (54873, 13)


In [10]:
df_import = df[df['Indicator Code'] == 'TMG_CIF_USD']
df_import.reset_index(drop=True, inplace=True)
df_import = df_import[required+years]
print('import shape = ', df_import.shape)

df_export = df[df['Indicator Code'] == 'TXG_FOB_USD']
df_export.reset_index(drop=True, inplace=True)
df_export = df_export[required+years]
print('export shape = ', df_export.shape)

import shape =  (29126, 12)
export shape =  (25747, 12)


In [12]:
df_import.sort_values(by=['Country Code', 'Counterpart Country Code'], inplace=True)
df_import.reset_index(drop=True, inplace=True)
df_import.to_excel('./resouces/processed/DOT_import_2013_2022.xlsx', index=False)

df_export.sort_values(by=['Country Code', 'Counterpart Country Code'], inplace=True)
df_export.reset_index(drop=True, inplace=True)
df_export.to_excel('./resouces/processed/DOT_export_2013_2022.xlsx', index=False)

In [13]:
cname_ccode_map

{'World': 1,
 'Emerging Market and Developing Economies by Source of Export Earnings: Fuel': 80,
 'Emerging Market and Developing Economies by Source of Export Earnings: Nonfuel': 92,
 'Advanced Economies': 110,
 'United States': 111,
 'United Kingdom': 112,
 'Austria': 122,
 'Belgium': 124,
 'Belgium-Luxembourg': 126,
 'Denmark': 128,
 'France': 132,
 'Germany': 134,
 'San Marino': 135,
 'Italy': 136,
 'Luxembourg': 137,
 'Netherlands': 138,
 'Norway': 142,
 'Sweden': 144,
 'Switzerland': 146,
 'Canada': 156,
 'Japan': 158,
 'Euro Area': 163,
 'Finland': 172,
 'Greece': 174,
 'Iceland': 176,
 'Ireland': 178,
 'Malta': 181,
 'Portugal': 182,
 'Spain': 184,
 'Turkey': 186,
 'Holy See': 187,
 'Yugoslavia': 188,
 'Australia': 193,
 'New Zealand': 196,
 'South African Common Customs Area (SACCA)': 198,
 'South Africa': 199,
 'Emerging and Developing Economies': 200,
 'Western Hemisphere': 205,
 'Argentina': 213,
 'Bolivia': 218,
 'Brazil': 223,
 'Chile': 228,
 'Colombia': 233,
 'Costa Rica

In [80]:
# df = df[required+years]
# df.sort_values(by=['Country Code', 'Counterpart Country Code'], inplace=True)
# df.reset_index(drop=True, inplace=True)

# df.to_excel('./resouces/processed/DOT_2013_2022.xlsx', index=False)

In [81]:
df = pd.read_excel('./resouces/processed/DOT_2013_2022.xlsx')
df

,Country Code,Counterpart Country Code,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,1,1,1.874810e+13,1.881840e+13,1.640550e+13,1.587800e+13,1.754940e+13,1.932910e+13,1.875490e+13,1.736720e+13,2.207400e+13,2.449870e+13
1,1,1,1.883500e+13,1.893840e+13,1.658520e+13,1.614960e+13,1.774740e+13,1.955330e+13,1.900180e+13,1.750240e+13,2.226740e+13,2.515010e+13
2,1,80,1.307330e+12,1.293960e+12,1.031950e+12,9.230170e+11,9.847160e+11,1.008510e+12,1.002190e+12,8.944610e+11,1.089760e+12,1.194150e+12
3,1,80,2.226700e+12,2.076360e+12,1.322770e+12,1.117610e+12,1.363280e+12,1.703360e+12,1.535570e+12,1.126620e+12,1.677990e+12,2.326700e+12
4,1,92,5.651010e+12,5.660180e+12,4.959090e+12,4.778770e+12,5.487020e+12,6.119090e+12,5.937230e+12,5.498820e+12,7.288310e+12,8.015630e+12
...,...,...,...,...,...,...,...,...,...,...,...,...
54868,998,967,1.629636e+08,1.259450e+08,1.145961e+08,7.887471e+07,1.001437e+08,1.139400e+08,1.452591e+08,1.897987e+08,2.978714e+08,2.943076e+08
54869,998,968,5.320919e+10,5.670467e+10,5.169727e+10,5.519190e+10,6.307040e+10,7.251561e+10,7.142027e+10,6.685897e+10,8.140859e+10,8.651352e+10
54870,998,968,3.902588e+10,4.379227e+10,3.929094e+10,4.242554e+10,4.886156e+10,5.706042e+10,5.460603e+10,4.825278e+10,5.913308e+10,6.499514e+10
54871,998,998,3.092400e+12,3.186790e+12,2.779370e+12,2.847290e+12,3.157040e+12,3.491270e+12,3.368380e+12,3.191920e+12,3.992830e+12,4.305840e+12


In [ ]:
# FIW data
# df = pd.read_excel('./resouces/downloads/FIW_2013-2023.xlsx', sheet_name='FIW13-23')
# df = df[['Country/Territory', 'Edition', 'Total']]
# df = df.pivot(index='Country/Territory', columns='Edition', values='Total')

# df.to_excel('./resouces/raw_data/fiw_2013_2023.xlsx')

# df_fiw = pd.read_excel('./resouces/processed/fiw_2013_2023.xlsx')
# df_fiw.head()

In [61]:
# not_available = []

# for country in df_fiw['Country Name'].values:
#     if country not in cname_ccode_map:
#         not_available.append(country)
#     else:
#         df_fiw.loc[df_fiw['Country Name'] == country, 'Country Code'] = int(cname_ccode_map[country])

# df_fiw.drop(['Country Name'], axis=1, inplace=True)
# df_fiw.dropna(inplace=True)
# df_fiw.reset_index(drop=True, inplace=True)
# df_fiw = df_fiw.astype(int)

# df_fiw.sort_values(by=['Country Code'], inplace=True)
# df_fiw.reset_index(drop=True, inplace=True)

# df_fiw.to_excel('./resouces/raw_data/fiw_2013_2023.xlsx', index=False)

df_fiw = pd.read_excel('./resouces/raw_data/fiw_2013_2023.xlsx')
df_fiw.head()

,Country Code,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,111,93,92,92,90,89,86,86,86,83,83,83
1,112,97,97,97,95,95,94,93,94,93,93,93
2,122,96,96,96,95,95,94,93,93,93,93,93
3,124,97,97,97,96,95,95,96,96,96,96,96
4,128,98,98,98,98,97,97,97,97,97,97,97
